In [ ]:
import numpy as np
import h5py
import ccfj
import matplotlib.pyplot as plt
import yaml
import os
from scipy.fft import fft2, ifft2, fftshift

In [ ]:
import sys
sys.path.append(r'../')
from toollib_standard import mathlib
from toollib_standard import plotlib

In [ ]:
with open('a-project.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
proj_name = proj['name']
proj_name = 'project/output_FJSJ_16-03/'
proj_name

In [ ]:
filename = proj_name+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = proj_name+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [ ]:
dir_stack = info_basic['dir_stack']
t = info_basic_bi['t']
f = info_basic_bi['f']
dt = 1/np.max(f)
t = (np.linspace(-len(f)-1,len(f)-1,2*(len(f)-1))+0.5)*dt/2

In [ ]:
key_subworks = info_basic['key_subworks']
#key_subworks.remove('23-08')
#key_subworks = ['07-03']
key_subworks

### f-k transformation

In [ ]:
def perform_2d_fft(data, t, r):
    """
    Perform 2D spatial discrete Fourier transform on input data.

    Parameters:
    - data: 2D array representing time-domain cross-correlation functions.
            Shape: (num_offsets, num_samples)
    - t: Time array.
    - r: Offset array.

    Returns:
    - kx: 1D array representing horizontal slowness.
    - ky: 1D array representing vertical slowness.
    - spectrum: 2D array representing the Fourier transformed data.
                Shape: (num_ky, num_kx)
    """

    # Calculate dimensions
    num_offsets, num_samples = data.shape

    # Calculate time and spatial intervals
    dt = t[1] - t[0]

    # Create frequency arrays
    freq_x = np.fft.fftshift(np.fft.fftfreq(num_samples, dt))
    freq_y = np.fft.fftshift(np.fft.fftfreq(num_offsets, r[1] - r[0]))

    # Perform 2D FFT
    spectrum = np.fft.fftshift(np.fft.fft2(data))

    return freq_x, freq_y, spectrum


In [ ]:
key_subwork = key_subworks[0]
outname = key_subwork+'_gather_linear.h5'
ncffile = h5py.File(dir_stack + outname,'r')
ncfs_sum_linear = ncffile['ncfs'][:]
ncfst_linear = mathlib.freq_time(ncfs_sum_linear)
r = ncffile['r'][:]
ncffile.close()

In [ ]:
for i in range(len(ncfst_linear)):
    ncfst_linear[i,:] = ncfst_linear[i,:]/np.max(ncfst_linear[i,:])

In [ ]:
t_new = t[np.abs(t)<=1]
data = ncfst_linear[:,np.abs(t)<=1]

In [ ]:
kx, ky, spectrum = perform_2d_fft(data,t,r)

In [ ]:
spectrum.shape

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.pcolor(t_new, r, data, cmap='viridis', shading='auto')
plt.colorbar()
plt.title('Original Data (2D Sine Function)')
plt.xlabel('Time (seconds)')
plt.ylabel('Offset (meters)')

# Plot the 2D Fourier transformed data with pcolor
plt.subplot(1, 2, 2)
plt.pcolor(kx,ky, np.log(1 + np.abs(spectrum)), cmap='viridis', shading='auto')
plt.colorbar()
plt.title('2D Spatial Fourier Transform')
plt.xlabel('kx (Horizontal Slowness)')
plt.ylabel('ky (Vertical Slowness)')
plt.xlim([-5,5])
plt.ylim([-5,5])
plt.tight_layout()
plt.show()

In [ ]:
plt.pcolor(t_new,r,data,cmap='jet')
# 设置x轴范围
plt.xlim([-2,2])
# y轴反转
#plt.gca().invert_yaxis()

In [ ]:
fig,ax = plt.subplots(1,2,figsize = (20,10))

start = info_basic['start']
interval = info_basic['interval']
flag_time = info_basic['flag_time']
xlim_T = [-1,1]
title1 = 'Linear'
ax[0] = plotlib.plot_ncfst(ax[0],t,ncfst_linear[start::interval],r[start::interval],title1,flag_time,xlim_T,0)
#ax[0].vlines(tag,0,0.2,linestyles='dashed',colors='g')
#ax[0].vlines(-tag,0,0.2,linestyles='dashed',colors='g')
ax[1] = plotlib.plot_ncfst(ax[1],t,ncfst_linear_remove[start::interval],r[start::interval],title1,flag_time,xlim_T,0)
#ax[1].vlines(tag,0,0.2,linestyles='dashed',colors='g')

plt.tight_layout()
plt.show()

In [ ]:
#remove_zorocor(v_tag,t0,a,proj_name,1)